The beginning of summarizing chapter 2: Nested models 

# Chapter 2: Nested models and bootstrap

Nested models; or when to stop adding more parameters;
- Akaike information criterion (AIC) and BIC
Bootstrap method for confidence intervals (Par 6.5 en 6.6 van lecture notes)
- Work out example with comparison to least-squares confidence intervals

Bayesian hierarchical modelling is a statistical model written in multiple levels (hierarchical form) that estimates the parameters of the posterior distribution using the Bayesian method.[1] The sub-models combine to form the hierarchical model, and Bayes' theorem is used to integrate them with the observed data and account for all the uncertainty that is present. The result of this integration is the posterior distribution, also known as the updated probability estimate, as additional evidence on the prior distribution is acquired. Source: Wikipedia hehe